# Proving the Playpen Agent/CodePlan for Compiler Errors

We need prove that the playpen code plan agents workflow will work as expected. This will be a way to run them and make sure that things work as we expect.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

sys.path.append("../../")

Obtaining file:///Users/shurley/repos/kai/kai
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/fabianvf/vcrpy.git (to revision httpx-async-threadpool) to /private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/pip-install-8g1r4qee/vcrpy_b49a8db815e444e3a7b4e330f623297f
  Running command git clone --filter=blob:none --quiet https://github.com/fabianvf/vcrpy.git /private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/pip-install-8g1r4qee/vcrpy_b49a8db815e444e3a7b4e330f623297f
  Running command git checkout -b httpx-async-threadpool --track origin/httpx-async-threadpool
  Switched to a new branch 'httpx-async-threadpool'
  branch 'httpx-async-threadpool' set up to track 'origin/httpx-async-threadpool'.
  Resolved https://github.com/fabianvf/vcrpy.git to commit db764f025eaad9459e1553da36fd71400a0c9849
 

In [2]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create the Test Environment

For creating the test envirnoment we will:

1. move the coolstore to a temp direcotry
2. Update the InventoryEntity to use UUID for the itemID as the type.

In [1]:
import tempfile
import shutil
import os
import subprocess

#First we need to download coolstore if it does not exist.

if not os.path.exists("./coolstore"):
    subprocess.run("../../example/fetch.sh")

temp_dir = tempfile.TemporaryDirectory()
coolstore_path = os.path.join(temp_dir.name, "coolstore")
shutil.copytree("./coolstore", coolstore_path)

## make the necessary change
shutil.copyfile("./testing_field_type_change_errors/InventoryEntity.java", f"{coolstore_path}/src/main/java/com/redhat/coolstore/model/InventoryEntity.java")

print(temp_dir)


Cloning into 'coolstore'...


<TemporaryDirectory '/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmpe89g9uvs'>


## Set up the Code Plan types and run code plan.

In [25]:
from playpen.repo_level_awareness.api import Agent, RpcClientConfig, Task, TaskResult, ValidationStep
from playpen.repo_level_awareness.codeplan import TaskManager
from playpen.repo_level_awareness.agents.compiler_agent import MavenCompilerAgent
from playpen.repo_level_awareness.maven_validator import MavenCompileStep
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig


config = RpcClientConfig(coolstore_path)

kai_config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(kai_config.models)


agent = MavenCompilerAgent(modelProvider.llm)

task_manager = TaskManager(
        config,
        None,
        validators=[MavenCompileStep(config)],
        agents=[agent],
    )


for task in task_manager.get_next_task():
    print(task)
    task_manager.supply_result(task_manager.execute_task(task))

TypeMismatchError(file='/private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmp6iv28juy/coolstore/src/main/java/com/redhat/coolstore/model/InventoryEntity.java', line=40, column=24, message='incompatible types: java.util.UUID cannot be converted to java.lang.String', details=[], parse_lines='[ERROR] /private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmp6iv28juy/coolstore/src/main/java/com/redhat/coolstore/model/InventoryEntity.java:[40,24] incompatible types: java.util.UUID cannot be converted to java.lang.String', expected_type=None, found_type=None)
TypeMismatchError(file='/private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmp6iv28juy/coolstore/src/main/java/com/redhat/coolstore/model/InventoryEntity.java', line=40, column=24, message='incompatible types: java.util.UUID cannot be converted to java.lang.String', details=[], parse_lines='[ERROR] /private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmp6iv28juy/coolstore/src/main/java/com/redhat/coolstore/model/Inven

## Cleanup temp dir

In [2]:
temp_dir.cleanup()